# de Sitter spacetime with lyra-geometry

This notebook reproduces the SageMath example `examples/research/sagemath-examples/SM_de_Sitter.ipynb`
using only the Python stack in lyra-geometry. We construct the de Sitter metric in hyperspherical
coordinates $(\tau, \chi, \theta, \phi)$, compute the Riemann and Ricci tensors, and verify the
constant-curvature relation and Einstein equation with $\Lambda = 3 b^2$.

The original SageMath notebook includes 3D plots of coordinate grids; those are omitted here
because we keep the notebook strictly Python-only.


In [1]:
import sympy as sp
import lyra_geometry as lg

sp.init_printing()


## Coordinates and embedding

We use the hyperspherical chart $(\tau, \chi, \theta, \phi)$ on $\mathcal{M}$ and embed it into
$\mathbb{R}^{1,4}$ with coordinates $(T, W, X, Y, Z)$. The embedding used in the SageMath notebook is



$\Phi(\tau,\chi,\theta,\phi) = (\sinh(b\tau)/b,\, \cosh(b\tau)\cos\chi/b,\, \cosh(b\tau)\sin\chi\sin\theta\cos\phi/b,\, \cosh(b\tau)\sin\chi\sin\theta\sin\phi/b,\, \cosh(b\tau)\sin\chi\cos\theta/b)$.


We compute the induced metric by pulling back the 5D Minkowski metric.


In [2]:
tau, chi, theta, phi = sp.symbols("tau chi theta phi", real=True)
b = sp.symbols("b", real=True, positive=True)

T = sp.sinh(b * tau) / b
W = sp.cosh(b * tau) * sp.cos(chi) / b
X = sp.cosh(b * tau) * sp.sin(chi) * sp.sin(theta) * sp.cos(phi) / b
Y = sp.cosh(b * tau) * sp.sin(chi) * sp.sin(theta) * sp.sin(phi) / b
Z = sp.cosh(b * tau) * sp.sin(chi) * sp.cos(theta) / b

embedding = sp.Matrix([T, W, X, Y, Z])
coords = sp.Matrix([tau, chi, theta, phi])

h = sp.diag(-1, 1, 1, 1, 1)
J = embedding.jacobian(coords)

metric_from_embedding = sp.simplify(J.T * h * J)
metric_from_embedding


⎡-1      0               0                       0             ⎤
⎢                                                              ⎥
⎢        2                                                     ⎥
⎢    cosh (b⋅τ)                                                ⎥
⎢0   ──────────          0                       0             ⎥
⎢         2                                                    ⎥
⎢        b                                                     ⎥
⎢                                                              ⎥
⎢                   2        2                                 ⎥
⎢                sin (χ)⋅cosh (b⋅τ)                            ⎥
⎢0       0       ──────────────────              0             ⎥
⎢                         2                                    ⎥
⎢                        b                                     ⎥
⎢                                                              ⎥
⎢                                       2       2        2     ⎥
⎢                        

We can also write the metric directly in closed slicing form as



$ds^2 = -d\tau^2 + (\cosh(b\tau)/b)^2 \left(d\chi^2 + \sin^2\chi (d\theta^2 + \sin^2\theta\, d\phi^2)\right)$.


We verify that it matches the induced metric computed above.


In [3]:
scale = sp.cosh(b * tau) / b
metric_closed = sp.diag(
    -1,
    scale**2,
    scale**2 * sp.sin(chi) ** 2,
    scale**2 * sp.sin(chi) ** 2 * sp.sin(theta) ** 2,
)

metric_check = sp.simplify(metric_from_embedding - metric_closed)
metric_check


⎡0  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎣0  0  0  0⎦

## de Sitter geometry with lyra-geometry

We now feed the metric into lyra-geometry and compute the Riemann tensor, Ricci tensor, and Ricci scalar.


In [6]:
space = lg.TensorSpace(coords=(tau, chi, theta, phi), metric=metric_closed)

Riemann = space.riemann
Ricci = space.ricci
R = space.ricci_scalar()



In [7]:
Riemann

In [8]:
Ricci

In [9]:
R

## Constant curvature check

For a 4D de Sitter spacetime of curvature scale $b$, the Riemann tensor satisfies



$R^i{}_{jlk} = \frac{R}{n(n-1)} (\delta^i{}_k g_{jl} - \delta^i{}_l g_{jk})$.


We check that all components simplify to zero for $R^i{}_{jlk} = - \frac{R}{6}g_{j[k}\delta^i{}_l$.


In [66]:
i, j, l, k = space.index("i j l k")

g = space.metric

n = space.dim

expected = space.tensor(
    (
        (R/(n*(n-1)))
        * (
            space.delta[+i, -k] * g[-j, -l]
            - space.delta[+i, -l] * g[-j, -k]
        )
    ),
    index=(+i,-j,-l,-k),
)

curvature_check = space.tensor(
    (Riemann[+i,-j,-l,-k] - expected[+i,-j,-l,-k]).fmt(),
    name="curvature_check"
)

rank = len(curvature_check.signature)

curvature_ok = all(
    [
        curvature_check[+i,-j,-l,-k](a,b,c,d) == 0
        for a in range(rank) for b in range(rank)
        for c in range(rank) for d in range(rank)
    ]
)
curvature_ok

True

## Einstein equation with $\Lambda = 3 b^2$

For de Sitter space, the Einstein equation in vacuum reads



$R_{ab} - \frac{1}{2} R g_{ab} - \Lambda g_{ab} = 0$ with $\Lambda = 3 b^2$.


We check that every component simplifies to zero.


In [69]:
m,n = space.index("m n")

Lambda = 3 * b ** 2

einstein_check = space.tensor(
    (
        Ricci[-m, -n]
        - sp.Rational(1, 2) * R * g[-m, -n]
        - Lambda * g[-m, -n]
    ),
    index=(-m, -n),
)

eisntein_ok = all(
    [
        einstein_check[-m, -n](a,b) == 0
        for a in range(rank) for b in range(rank)
    ]
)

eisntein_ok


True